# Examen práctico del primer parcial

Utiliza solo librerías de álgebra matricial como numpy y scipy. Librerías de visualización y cualquier función que sea parte del core de Python.

In [ ]:
import numpy as np
from sklearn.datasets import load_digits

## 1 Implementa una neurona con sus componentes básicos.

In [ ]:
class Neurona00():
    def __init__(self, tamanio_entrada):
        self.pesos = np.random.rand(tamanio_entrada)
        self.bias = np.random.rand()

class Neurona01(Neurona00):
    def valor_neto(self,entrada):
        return np.matmul(entrada, self.pesos) + self.bias

class Neurona02(Neurona01):

    def __init__(self, tamanio_entrada, funcion_activacion):
        self.pesos = np.random.rand(tamanio_entrada)
        self.bias = np.random.rand()
        self.funcion_activacion = funcion_activacion

    def salida(self,entrada):
        return self.funcion_activacion(self.valor_neto(entrada))


## 1.1 Enumera cuales son estos componentes

### 1.1.1 Las entradas
Es la cantidad de señales que va a recibir la neurona

Depende de la dimensionalidad de nuestro problema, en este caso son imágenes de 8x8 por lo que tenemos 64 entradas.

### 1.1.2 Los pesos
Debemos de tener un peso para cada una de las entradas, lo que si podemos elegir es como se inicializan estos pesos. Algunas opciones son

1. Aleatorios
2. iniciales en 0
3. Determinados de manera fija por conocimiento del problema

Se elige inicializarlo de manera aleatoria ya que no se cuenta con información previa.

### 1.1.3. El sesgo o bias
Es un peso que no esta asociado a ninguna entrada, al igual que los pesos tiene varias maneras de inicializase.

### 1.1.4. Valor neto
Es la función que integra todas las entradas y las condensa en un solo numero. Aunque existen otras implementaciones la mas común es la lineal la cual consiste en multiplicar cada entrada por su peso asociado y sumar el bias.

### 1.1.5 La Función de Activación
Es la fución encargada de definir si la neurona se activa o no.

Para esto hay una gran selección de funciones

- Sigmoide: Utiliza la función sigmoide, que produce una salida en el rango (0, 1).
- ReLU: Emplea la función ReLU, que produce una salida lineal si es positiva y cero si es negativa.
- Tangente hiperbólica: Utiliza la función tanh, que produce una salida en el rango (-1, 1).
- Función lineal: Utiliza una función lineal que en realidad pasa el valor como es
Como este problema es de clasificación la selección mas segura es la función sigmoide.



# 2 Agrega a la implementación necesaria para entrenar la neurona         


In [ ]:
class Neurona03(Neurona02):

    def compila(self,funcion_perdida,derivada_funcion_perdida_pesos,derivada_funcion_perdida_bias):
        self.funcion_perdida = funcion_perdida
        self.derivada_funcion_perdida_pesos = derivada_funcion_perdida_pesos
        self.derivada_funcion_perdida_bias = derivada_funcion_perdida_bias


    def calcula_error(self,entradas,y_real):
        return self.funcion_perdida([self.salida(entradas)] ,y_real)

    def decenso_gradiente_step(self, X, y):
        # todas las variables son vectores (n muestras) excepto total_error
        valor_neto = self.valor_neto(X)
        y_pred = self.funcion_activacion(valor_neto)
        total_error = self.funcion_perdida(y,y_pred)
        gradiente_pesos = self.derivada_funcion_perdida_pesos(X,y,y_pred)
        gradiente_bias = self.derivada_funcion_perdida_bias(X,y,y_pred)

        return gradiente_pesos, gradiente_bias, total_error

    def desenso_gradiente(self,epochs, X, y, tasa_aprendizaje):
        lista_error = []
        for epoch in range(epochs):
            gradiente_pesos, gradiente_bias, total_error = self.decenso_gradiente_step( X, y)
            self.pesos = self.pesos - (tasa_aprendizaje * gradiente_pesos)
            self.bias = self.bias - (tasa_aprendizaje * gradiente_bias)
            lista_error.append(total_error)
            if (epoch+1) % int(epochs/20) == 0:
                print(f"Epoch {epoch+1}, Error: {total_error.mean()}, {self.pesos[:5]} {self.bias}")
        return lista_error

## 2.1 De la misma manera que en el punto anterior, enumera cuales son estos componentes, para cada uno

### Función de Pérdida (Loss Function):

Una función que mide la discrepancia entre la salida real de la neurona y la salida esperada.

**Regresión:**

- [Error Cuadrático Medio](https://keras.io/api/losses/regression_losses/#meansquarederror-class) (MSE): Calcula el promedio de los cuadrados de las diferencias entre las predicciones y los valores reales.
- [Error Absoluto Medio](https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class) (MAE): Calcula el promedio de las diferencias absolutas entre las predicciones y los valores reales.
- [Error Huber](https://keras.io/api/losses/regression_losses/#huber-class): Combina las propiedades de MSE y MAE.

**Clasificación**

- [Entropía Cruzada Binaria](https://keras.io/api/losses/probabilistic_losses/#binarycrossentropy-class) (Binary Cross-Entropy): Es utilizada para medir la discrepancia entre las probabilidades predichas y las etiquetas verdaderas.
- [Entropía Cruzada Categórica](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class) (Categorical Cross-Entropy): Extensión de la entropía cruzada para clasificación multiclase.
- [Hinge Loss](https://keras.io/api/losses/hinge_losses/#hinge-class) : Utilizada en máquinas de soporte vectorial (SVM) para problemas de clasificación binaria. Es especialmente útil cuando se desea maximizar el margen entre las clases.


Al ser un problema de clasificación binaria lo mas adecuado y simple es implementar Binary Cross-Entropy



### Optimizador:

Un algoritmo que ajusta los pesos y el sesgo de la neurona para minimizar la función de pérdida.


- [SGD](https://keras.io/api/optimizers/sgd/): Descenso de gradiente con momento.
- [RMSprop](https://keras.io/api/optimizers/rmsprop/): Utiliza medias móviles del gradiente para ajustar la tasa de aprendizaje.
- [Adam](https://keras.io/api/optimizers/adam/): Descenso de gradiente estocástico que adapta la tasa de aprendizaje con los momentos estadísticos.
- [AdamW](https://keras.io/api/optimizers/adamw/):
- [Adadelta](https://keras.io/api/optimizers/adadelta/): Basado en Adam usa una tasa de aprendizaje para cada dimensión respecto a la frecuencia de actualización en una ventana.
- [Adagrad](https://keras.io/api/optimizers/adagrad/): Basado en Adam usa una tasa de aprendizaje para cada dimensión respecto a la frecuencia de actualización en el entrenamiento.
- [Adamax](https://keras.io/api/optimizers/adamax/): Tiene la capacidad de adaptar tasa de aprendizaje basado en las características de los datos. Por ejemplo, datos de habla con ruido variable.
- [Adafactor](https://keras.io/api/optimizers/adafactor/): Solo guarda información parcial de los gradientes pasados.
- [Nadam](https://keras.io/api/optimizers/Nadam/): Adam con momento de Nesterov.
- [Ftrl](https://keras.io/api/optimizers/ftrl/): Adaptado para modelos espacios de características muy grandes.

Por sencillez se opta por implementar descenso de gradiente por batches.




# 3 Utiliza tu implementación para clasificar la base de datos [digits](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits) de sklearn

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## 3.1 Utiliza solo dos clases, seleccionadas de manera aleatoria.         


In [ ]:
tX, ty = load_digits(return_X_y=True)
# Generar el primer número aleatorio entre 0 y 9
num1 = np.random.randint(0, 9)  # Genera un número aleatorio entre 10 y 99
# Generar el segundo número aleatorio diferente al primero
num2 = num1
while num2 == num1:
    num2 = np.random.randint(0, 9)

#filtramos los registros con las 2 clases aleatorias
fy = ty[(ty ==num1) |(ty ==num2)]
fX = tX[(ty ==num1) |(ty ==num2)]

print(num1,num2)

## 3.2 Realiza los pasos de pre-procesamiento que consideres necesarios para mejorar la calidad del clasificador.        


In [ ]:
X_ = (fX-fX.mean())/fX.std()
# Estandarizar restar la media y dividir entre desviación estandar 
# (datos centrados en 0 con desviación de 1)


In [ ]:
mask_num1 = fy==num1
mask_num2 = fy==num2
y_ = fy
y_[mask_num1] = 0
y_[mask_num2] = 1

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2)

3.3 Entrena la neurona para predecir cuál es el número de la imagen.

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def binary_cross_entropy(y, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

def binary_cross_entropy_derivada_pesos(X, y, y_pred):
    return np.matmul(X.T,(y_pred-y))/len(X)

def binary_cross_entropy_derivada_bias(X, y, y_pred):
    return sum(y_pred-y)/len(X)

In [ ]:
neurona = Neurona03(64,sigmoid)
neurona.compila(
    binary_cross_entropy,
    derivada_funcion_perdida_pesos=binary_cross_entropy_derivada_pesos,
    derivada_funcion_perdida_bias=binary_cross_entropy_derivada_bias
    )
ta = 0.1

In [ ]:
perdida = neurona.desenso_gradiente(epochs=10000, X=X_train, y=y_train, tasa_aprendizaje=ta)

## 3.4 Explica las razones detrás de la selección de los distintos parámetros

### 3.4.1 Epochs
Se incrementaron de manera gradual hasta que la perdida se disminuye muy poco entre epochs. Antes de que se estanque por completo porque puede provocar sobre ajuste.

#3.4.2 Tamaño de batch
Se opta por un tamaño de batch completo ya que no son muchos datos para asegurar estabilidad

#3.4.3 Criterio de alto
El unico criterio de alto es el número de epochs

#3.4.3 Tasa de aprendizaje

Utilizamos una tasa de aprendizaje lo mas grande que no provoque aumentos en la perdida a través de los epochs




## 3.5 Obtén el valor de la pérdida (loss) y da una explicación de cuando y porque cambian estos valores (o porque no)

Vemos que la perdida disminuye de manera correcta, muy rápido al principio y mas lento al final. Hay que tomar en cuenta que la escala es logarítmica. Por lo que podríamos empezar a decir que el aprendizaje comienza a estancarse


In [ ]:
plt.plot(perdida) #perdida
plt.yscale('log')

## 3.6 Evalúa el desempeño contra una muestra en la que nunca se haya entrenado.        
### 3.6.1 Da los principales indicadores de calidad          
### 3.6.2 Interpreta cuál es el significado cada valor de los indicadores    

In [ ]:
salida_test = neurona.salida(X_test)
mask = salida_test < 0.5
prediccion = np.zeros_like(salida_test)
prediccion[salida_test > 0.5 ] = 1
prediccion = prediccion.astype(int)
prediccion

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


conf_matrix = confusion_matrix(y_test, prediccion)
accuracy = accuracy_score(y_test, prediccion)
precision = precision_score(y_test, prediccion)
recall = recall_score(y_test, prediccion)

print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Vemos que la matriz de confusión es perfecta. De la misma manera accuracy presision y recall.

Vemos que no hubo sobreentrenamiento.


# 4 Presenta una conclusión final del modelo creado. Cuales son las fortalezas y debilidades, así como sugerencias para mejorar su desempeño.
para este set de datos el modelo resulto muy bueno. Aunque la cantidad de epochs podría ser excesiva, resulta peligroso aumentar la tasa de aprendizaje.

Se puede mejorar el modelo con un monitoreo de métricas que nos permitan detener de manera anticipada el entrenamiento.
